<a href="https://colab.research.google.com/github/arica-ee/Bike_Share/blob/main/bike_route.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 處理資料、載入套件

In [62]:
# !pip install osmnx
# !pip install --upgrade osmnx

In [63]:
from google.colab import output
output.enable_custom_widget_manager()

In [64]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from IPython.display import display
import folium
from folium.plugins import MarkerCluster

In [65]:
# 載入資料
!gdown 1-f1PqQ1V2eYrCJ8c2FPBmcXTO_ewbqJX -O trip.csv
!gdown 1jPAtOX2tnfAYw0U5kYWimHNW_FqOXAge -O station.csv

trip = pd.read_csv("trip.csv", dtype={"zip_code": str})
station = pd.read_csv("station.csv")

Downloading...
From (original): https://drive.google.com/uc?id=1-f1PqQ1V2eYrCJ8c2FPBmcXTO_ewbqJX
From (redirected): https://drive.google.com/uc?id=1-f1PqQ1V2eYrCJ8c2FPBmcXTO_ewbqJX&confirm=t&uuid=653cbb48-f26f-4fcc-abf3-d6df84b71b53
To: /content/trip.csv
100% 121M/121M [00:01<00:00, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jPAtOX2tnfAYw0U5kYWimHNW_FqOXAge
To: /content/station.csv
100% 5.65k/5.65k [00:00<00:00, 15.8MB/s]


In [66]:
# 處理資料格式
station.rename(columns = {"name" : "station_name"}, inplace = True)
trip["start_date"] = pd.to_datetime(trip["start_date"])
# 統整各站點用戶借車數
borrow = trip.start_station_id.value_counts().sort_index()

In [67]:
center = [round(station.lat.mean(), 2), round(station.long.mean(), 2)]
print(f"緯度從{station.lat.max()}到{station.lat.min()}，中心點是：{center[0]}")
print(f"經度從{station.long.max()}到{station.long.min()}，中心點是：{center[1]}")

緯度從37.80477到37.329732，中心點是：37.59
經度從-121.877349到-122.418954，中心點是：-122.22


# **各站點借車熱力圖**

In [68]:
def hot_map(start_time, end_time, start_weekday, end_weekday):
  # 設定底圖
  bike_map = folium.Map(location = [center[0], center[1]], zoom_start = 10, control_scale = True) # control_scale:比例尺

  # 依「城市」標出站點顏色
  for i in range(0,len(station)):
    if station.loc[i].city == "San Francisco":
      color = "purple"
    elif station.loc[i].city == "San Jose":
      color = "green"
    elif station.loc[i].city == "Mountain View":
      color = "blue"
    elif station.loc[i].city == "Palo Alto":
      color = "black"
    else:
      color = "red"

    # 標出站點
    circle = folium.CircleMarker(
        location = [station.iloc[i].lat, station.iloc[i].long],
        radius = borrow.iloc[i] / 500, # 設定半徑
        tooltip = f"{station.iloc[i].id}  {station.iloc[i].station_name}", # 設定標籤
        fill = True, # 填滿顏色
        color = color, # 設定顏色
        )
    circle.add_to(bike_map)

  display(bike_map)
  bike_map.save("bike_map.html")
  print("地圖已儲存為 bike_map.html")

# **路線圖**

In [69]:
# 創建新的欄位表示路線
def route_map(start_time, end_time, start_weekday, end_weekday, route_count):

  SpecDay = trip.loc[trip.start_date.dt.hour.between(start_time, end_time) & trip.start_date.dt.weekday.between(start_weekday, end_weekday)].copy()
  SpecDay['開始到結束路線'] = SpecDay['start_station_id'].astype(str) + ' ' + SpecDay['end_station_id'].astype(str)

  # 計算每條路線出現的次數，並取出特定數量熱門路線
  route_counts = SpecDay['開始到結束路線'].value_counts().nlargest(route_count)

  # 拆分路線並合併計數
  rank_route = route_counts.rename_axis("route").reset_index(name="count")
  rank_route[["start_id", "end_id"]] = rank_route["route"].str.split(expand=True)
  rank_route["count"] = rank_route["count"].astype(int)

  central_point = [center[0], center[1]]
  # 建一個以”單車“為交通方式的道路網
  G = ox.graph_from_point(central_point, dist=25000, network_type='bike')
  bike_map = folium.Map(location = central_point, zoom_start = 10, control_scale = True) # control_scale:比例尺

  # 標出熱門圈圈
  for i in range(0,len(station)):
    if station.loc[i].city == "San Francisco":
      color = "purple"
    elif station.loc[i].city == "San Jose":
      color = "green"
    elif station.loc[i].city == "Mountain View":
      color = "blue"
    elif station.loc[i].city == "Palo Alto":
      color = "black"
    else:
      color = "red"

    # 標出站點
    marker_cluster = MarkerCluster().add_to(bike_map)

    marker = folium.Marker(
      location=[station.iloc[i].lat, station.iloc[i].long],
      tooltip=f"{station.iloc[i].id}  {station.iloc[i].station_name}",  # 滑鼠懸停時顯示
      icon=folium.Icon(color=color)  # 設定標記顏色
    )
    marker.add_to(marker_cluster)  # 加入聚合群組

  # 畫出特定數量的熱門路線，並依照熱門程度區分顏色、線條粗細
  rank_route["weight"] = np.interp(
    rank_route["count"],
    [min(rank_route["count"]), (min(rank_route["count"])+ max(rank_route["count"])) / 3 , (min(rank_route["count"])+ max(rank_route["count"]))* 2 / 3, max(rank_route["count"])],
    [1, 4, 7, 10]
  )

  for i in rank_route.index:
    if rank_route.loc[i]["weight"] >= 10:
      weight = 10
      color = "purple"
    elif rank_route.loc[i]["weight"] >= 7:
      weight = 4
      color = "green"
    elif rank_route.loc[i]["weight"] >= 4:
      weight = 3
      color = "blue"
    else:
      weight = 2
      color = "black"
    start_id = rank_route.loc[i].start_id
    end_id = rank_route.loc[i].end_id

    # 獲取起點、終點的經緯度
    start_location = (
        station[station["id"] == int(start_id)].lat.values[0],
        station[station["id"] == int(start_id)].long.values[0]
    )
    end_location = (
        station.loc[station['id'] == int(end_id)].lat.values[0],
        station.loc[station['id'] == int(end_id)].long.values[0]
    )

    # 計算地圖上的最近點
    start_node = ox.distance.nearest_nodes(G, start_location[1], start_location[0])
    end_node = ox.distance.nearest_nodes(G, end_location[1], end_location[0])
    shortest_path = nx.shortest_path(G, source=start_node, target=end_node, weight='length')

    # 取得最近點並標示在地圖上
    path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]

    # 標出路線
    folium.PolyLine(locations=path_coords,
                    tooltip = f"{i+1}th popular route\n{start_id} - {end_id}",
                    color=color,
                    weight=weight).add_to(bike_map)

  display(bike_map)
  bike_map.save("bike_map.html")
  print("地圖已儲存為 bike_map.html")


# **熱力圖＋路線圖**

In [70]:
# 創建新的欄位表示路線
def hot_route(start_time, end_time, start_weekday, end_weekday, route_count):

  SpecDay = trip.loc[trip.start_date.dt.hour.between(start_time, end_time) & trip.start_date.dt.weekday.between(start_weekday, end_weekday)].copy()
  SpecDay['開始到結束路線'] = SpecDay['start_station_id'].astype(str) + ' ' + SpecDay['end_station_id'].astype(str)

  # 計算每條路線出現的次數，並取出特定數量熱門路線
  route_counts = SpecDay['開始到結束路線'].value_counts().nlargest(route_count)

  # 拆分路線並合併計數
  rank_route = route_counts.rename_axis("route").reset_index(name="count")
  rank_route[["start_id", "end_id"]] = rank_route["route"].str.split(expand=True)
  rank_route["count"] = rank_route["count"].astype(int)

  central_point = [center[0], center[1]]
  # 建一個以”單車“為交通方式的道路網
  G = ox.graph_from_point(central_point, dist=25000, network_type='bike')
  bike_map = folium.Map(location = central_point, zoom_start = 10, control_scale = True) # control_scale:比例尺

  # 標出熱門圈圈
  for i in range(0,len(station)):
    if station.loc[i].city == "San Francisco":
      color = "purple"
    elif station.loc[i].city == "San Jose":
      color = "green"
    elif station.loc[i].city == "Mountain View":
      color = "blue"
    elif station.loc[i].city == "Palo Alto":
      color = "black"
    else:
      color = "red"

    # 標出站點
    circle = folium.CircleMarker(
        location = [station.iloc[i].lat, station.iloc[i].long],
        radius = borrow.iloc[i] / 500, # 設定半徑
        tooltip = f"{station.iloc[i].id}  {station.iloc[i].station_name}", # 設定標籤
        fill = True, # 填滿顏色
        color = color, # 設定顏色
        )
    circle.add_to(bike_map)

  # 畫出特定數量的熱門路線，並依照熱門程度區分顏色、線條粗細
  rank_route["w"] = np.interp(
    rank_route["count"],
    [min(rank_route["count"]), (min(rank_route["count"])+ max(rank_route["count"])) / 3 , (min(rank_route["count"])+ max(rank_route["count"]))* 2 / 3, max(rank_route["count"])],
    [1, 4, 7, 10]
  )

  for i in rank_route.index:
    if rank_route.loc[i]["w"] >= 8:
      weight = 10
      color = "purple"
    elif rank_route.loc[i]["w"] >= 5:
      weight = 7
      color = "green"
    elif rank_route.loc[i]["w"] >= 2:
      weight = 2
      color = "blue"
    else:
      weight = 1
      color = "black"
    start_id = rank_route.loc[i].start_id
    end_id = rank_route.loc[i].end_id

    # 獲取起點、終點的經緯度
    start_location = (
        station[station["id"] == int(start_id)].lat.values[0],
        station[station["id"] == int(start_id)].long.values[0]
    )
    end_location = (
        station.loc[station['id'] == int(end_id)].lat.values[0],
        station.loc[station['id'] == int(end_id)].long.values[0]
    )

    # 計算地圖上的最近點
    start_node = ox.distance.nearest_nodes(G, start_location[1], start_location[0])
    end_node = ox.distance.nearest_nodes(G, end_location[1], end_location[0])
    shortest_path = nx.shortest_path(G, source=start_node, target=end_node, weight='length')

    # 取得最近點並標示在地圖上
    path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]

    # 標出路線
    folium.PolyLine(locations=path_coords,
                    tooltip = f"{i+1}th popular route. {start_id} - {end_id}",
                    color=color,
                    weight=weight).add_to(bike_map)

  display(bike_map)
  bike_map.save("bike_map.html")
  print("地圖已儲存為 bike_map.html")


# 讓使用者選擇繪製何種圖表及時間範圍

In [71]:
while True:
  choice = input("請選擇要畫(1)熱力圖、(2)路線圖、(3)或是熱力＋路線圖, 或按任意鍵停止畫圖")
  if choice == "1" or choice == "2" or choice == "3":
    seq = []
    seq.append(input("請輸入開始時間（24小時制）"))
    seq.append(input("請輸入結束時間（24小時制）"))
    seq.append(input("請輸入從星期幾開始（星期一請輸入1，以此類推）"))
    seq.append(input("請輸入從星期幾結束（星期一請輸入1，以此類推）"))

    if choice == "1":
      hot_map(int(seq[0]), int(seq[1]), int(seq[2])-1, int(seq[3])-1)
    elif choice == "2":
      route_count = int(input("請輸入需要印出幾條熱門路線"))
      route_map(int(seq[0]), int(seq[1]), int(seq[2])-1, int(seq[3])-1, route_count)
    elif choice == "3":
      route_count = int(input("請輸入需要印出幾條熱門路線"))
      hot_route(int(seq[0]), int(seq[1]), int(seq[2])-1, int(seq[3])-1, route_count)
  else:
    break;

請選擇要畫(1)熱力圖、(2)路線圖、(3)或是熱力＋路線圖, 或按任意鍵停止畫圖3
請輸入開始時間（24小時制）12
請輸入結束時間（24小時制）18
請輸入從星期幾開始（星期一請輸入1，以此類推）1
請輸入從星期幾結束（星期一請輸入1，以此類推）5
請輸入需要印出幾條熱門路線20


地圖已儲存為 bike_map.html
請選擇要畫(1)熱力圖、(2)路線圖、(3)或是熱力＋路線圖, 或按任意鍵停止畫圖q
